<a href="https://colab.research.google.com/github/BrainConnection/Dynamical_Decoupling_Project/blob/main/problem2_qiskit_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit
!pip install qiskit-ibm-runtime
!pip install numpy
!pip install matplotlib
!pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━

# test

In [3]:
import qiskit
import qiskit_ibm_runtime
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator, Batch

In [5]:
num_qubits = 10

In [6]:
def circuit_base(params, num_qubits):
    qc = QuantumCircuit(num_qubits)
    for w in range(num_qubits):
        qc.h(w)
        qc.ry(params[w], w)
    for w in range(num_qubits-1):
        qc.cx(w, w + 1)
    for w in range(num_qubits):
        qc.rz(params[w + num_qubits], w)
    return qc

In [7]:
params = np.random.randn(2 * num_qubits)
qc = circuit_base(params, num_qubits)
%matplotlib inline

qc.draw("mpl")

In [10]:
def generate_pauli_string(location, pauli, num_qubits):
    string = ["I"] * num_qubits
    for i, loc in enumerate(location):
        string[num_qubits-loc-1] = pauli[i]

    result = ''.join(char for char in string)

    return result


In [11]:
def calculate_classical_shadow(circuit, shadow_size, backend):
    num_qubits = circuit.num_qubits

    unitary_ensemble = ["X", "Y", "Z"]

    unitary_ids = np.random.randint(0, 3, size=(shadow_size, num_qubits))
    outcomes = np.zeros((shadow_size, num_qubits))

    pm = generate_preset_pass_manager(backend = backend, optimization_level=1)
    isa_circuit = pm.run(circuit)

    circuits = [isa_circuit] * shadow_size

    obs_list = []

    for ns in range(shadow_size):
        obs = SparsePauliOp(generate_pauli_string(range(num_qubits),[unitary_ensemble[int(unitary_ids[ns, i])] for i in range(num_qubits)],num_qubits)).apply_layout(isa_circuit.layout)
        obs_list.append(obs)


    max_circuits = 1000
    all_partitioned_circuits = []
    all_partitioned_obs = []
    for i in range(0, len(circuits), max_circuits):
        all_partitioned_circuits.append(circuits[i : i + max_circuits])
        all_partitioned_obs.append(obs_list[i:i+max_circuits])

    jobs = []


    with Batch(backend=backend) as batch:
        estimator = Estimator(batch)
        for job_idx, (partitioned_circuits, partitioned_obs) in enumerate(zip(all_partitioned_circuits, all_partitioned_obs)):
            print("Job Running"+ " " + str(job_idx))
            pub = [(circuit, obs) for circuit, obs in zip(partitioned_circuits, partitioned_obs)]
            job = estimator.run(pub)
            result = job.result()
            print("Job Run" + " " + str(job_idx) )
            for idx, pub_result in enumerate(result):
                exp_val = pub_result.data.evs
                outcomes[job_idx*max_circuits + idx :] = exp_val
            jobs.append(job)
            job.status()


    return (outcomes, unitary_ids)

In [12]:
list_of_observables = ([SparsePauliOp(generate_pauli_string([i,i+1],["X","X"], num_qubits)) for i in range(num_qubits - 1)]  +[SparsePauliOp(generate_pauli_string([i,i+1],["Y","Y"], num_qubits)) for i in range(num_qubits - 1)] +[SparsePauliOp(generate_pauli_string([i,i+1],["Z","Z"], num_qubits))for i in range(num_qubits - 1)] )

In [13]:
SparsePauliOp("XXIIIIIZZZX").to_list(array=True)[0][0]

'XXIIIIIZZZX'

In [14]:
def estimate_shadow_observable(shadow, observable, k=10):
    """
    Adapted from https://github.com/momohuang/predicting-quantum-properties
    Calculate the estimator E[O] = median(Tr{rho_{(k)} O}) where rho_(k)) is set of k
    snapshots in the shadow. Use median of means to ameliorate the effects of outliers.

    Args:
        shadow (tuple): A shadow tuple obtained from `calculate_classical_shadow`.
        observable (qml.Observable): Single PennyLane observable consisting of single Pauli
            operators e.g. qml.PauliX(0) @ qml.PauliY(1).
        k (int): number of splits in the median of means estimator.

    Returns:
        Scalar corresponding to the estimate of the observable.
    """
    shadow_size, num_qubits = shadow[0].shape



    def find_not_I(observable):
        not_I = []
        string = observable.to_list(array=True)[0][0]
        for i, c in enumerate(string):
            if c != "I":
                not_I.append((num_qubits-i-1,c))
        return not_I



    map_name_to_int = {"X": 0, "Y": 1, "Z": 2}
    target_obs, target_locs = np.array([map_name_to_int[tuple[1]] for tuple in find_not_I(observable=observable)]), np.array([tuple[0] for tuple in find_not_I(observable=observable)])



    # classical values
    b_lists, obs_lists = shadow
    means = []

    # loop over the splits of the shadow:
    for i in range(0, shadow_size, shadow_size // k):

        # assign the splits temporarily
        b_lists_k, obs_lists_k = (
            b_lists[i : i + shadow_size // k],
            obs_lists[i : i + shadow_size // k],
        )

        # find the exact matches for the observable of interest at the specified locations
        indices = np.all(obs_lists_k[:, target_locs] == target_obs, axis=1)

        # catch the edge case where there is no match in the chunk
        if sum(indices) > 0:
            # take the product and sum
            product = np.prod(b_lists_k[indices][:, target_locs], axis=1)
            means.append(np.sum(product) / sum(indices))
        else:
            means.append(0)

    return np.median(means)

In [26]:
!pip install qiskit-aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 62.0 MB/s eta 0:00:00


In [27]:
from qiskit_aer import AerSimulator

In [28]:
"""
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q-skku/hackathon-2024/hackathon',
    token='24ce54238a7bb29fe5e3eb450acefed2444169ce5e525f08f9515676b1cf257ea65b85895d4157a30359eb3d9a4aab9873008d7edf6ff177ed29015e625e4ad6'
)
"""
QiskitRuntimeService.save_account(channel="ibm_quantum",
                                  token="a0ebb10e52440a32a2711b5952c70f94cc00a77a99442a4805e461ffc889b7e3b87af8dc3e30887b04f879a660bf5177234b71653452ec6c0078b8f46b2076fb",
                                  set_as_default=True,
                                  overwrite=True)

service = QiskitRuntimeService()


#backend = service.least_busy(operational=False, simulator=True)

backend = AerSimulator()

In [29]:
print(backend)

AerSimulator('aer_simulator')


In [30]:
# make a array with exponential values
shadow_sizes = [1000, 2000, 3000, 4000, 5000]
estimates = []
circuit = circuit_base(params, num_qubits=num_qubits)
for shadow_size in shadow_sizes:
    shadow = calculate_classical_shadow(circuit, int(shadow_size), backend)
    # shadow = calculate_classical_shadow(circuit, int(shadow_size), backend)
    estimates.append([estimate_shadow_observable(shadow, o) for o in list_of_observables])

Job Running 0
Job Run 0
Job Running 0
Job Run 0
Job Running 1
Job Run 1
Job Running 0
Job Run 0
Job Running 1
Job Run 1
Job Running 0
Job Run 0
Job Running 1
Job Run 1
Job Running 2
Job Run 2
Job Running 0
Job Run 0
Job Running 1
Job Run 1
Job Running 2
Job Run 2
Job Running 3
Job Run 3
Job Running 0
Job Run 0
Job Running 1
Job Run 1
Job Running 2
Job Run 2
Job Running 3
Job Run 3
Job Running 4
Job Run 4
Job Running 5
Job Run 5
Job Running 0
Job Run 0
Job Running 1
Job Run 1
Job Running 2
Job Run 2
Job Running 3
Job Run 3
Job Running 4
Job Run 4
Job Running 5
Job Run 5
Job Running 6
Job Run 6
Job Running 7
Job Run 7
Job Running 0
Job Run 0
Job Running 1
Job Run 1
Job Running 2
Job Run 2
Job Running 3
Job Run 3
Job Running 4
Job Run 4
Job Running 5
Job Run 5
Job Running 6
Job Run 6
Job Running 7
Job Run 7
Job Running 8
Job Run 8
Job Running 9
Job Run 9
Job Running 10
Job Run 10
Job Running 0
Job Run 0
Job Running 1
Job Run 1
Job Running 2
Job Run 2
Job Running 3
Job Run 3
Job Running 4


In [31]:
estimates

[[0.00032088160514831543,
  0.00032463669776916504,
  0.0003904998302459717,
  0.0003928810358047485,
  0.00022199749946594238,
  0.000421866774559021,
  0.00023080905278523764,
  0.00022482872009277344,
  0.00046062469482421875,
  0.0003107150395711263,
  0.00027562777201334634,
  0.00024405717849731445,
  0.0004241636821201869,
  0.0003361264864603678,
  0.0004037817319234212,
  0.004333910014894273,
  0.0004437656113595674,
  0.00040747902610085225,
  0.00028056502342224123,
  0.0004005829493204752,
  0.00028519971030099053,
  0.0003924846649169922,
  0.00018640756607055665,
  0.0001612106959025065,
  0.0005340894063313802,
  0.0002583333424159459,
  0.0003711846139695909],
 [0.0003801584243774414,
  0.0003601710001627604,
  0.0007113023237748579,
  0.0006772364888872419,
  0.00035100777943929037,
  0.00043140487237410116,
  0.000591576099395752,
  0.0002658843994140625,
  0.0013872981071472168,
  0.00023439271109444753,
  0.0003885843537070534,
  0.0011811190181308323,
  0.00065066